In [17]:
import cv2
import os
import numpy as np
import pandas as pd
#from matpliotlib import pyplot as plt
import tensorflow as tf
import zipfile

In [18]:

zip_path = 'dados\Datasets\cat_dog_1.zip'
zip_file = zipfile.ZipFile(zip_path, 'r')
zip_file.extractall('./')



In [19]:
train_path = './cat_dog_1/train'
train_files = [os.path.join(train_path, f) for f in os.listdir(train_path)]
train_imagens = []
classes = [] # 0 - gato, 1 - cachorro

for f in train_files:
    img = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (250, 250))
    img = img.ravel()
    train_imagens.append(img)
    class_name = os.path.basename(f).split('\\')[-1].split('.')[0]

    if 'cat' in class_name:
        classes.append(0)
    else:
        classes.append(1)
train_imagens = np.asarray(train_imagens)
classes = np.asarray(classes)
train_imagens.shape, classes.shape

total_per_class = np.unique(classes, return_counts=True)
total_per_class


(array([0, 1]), array([2000, 2000], dtype=int64))

In [20]:
train_imagens[0].shape
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(train_imagens)
print(X_scaled.min(), X_scaled.max())





0.0 1.0


In [21]:
from sklearn.model_selection import train_test_split

x_train, _, y_train, _ = train_test_split(X_scaled, classes, test_size=0.3, random_state=42, stratify=classes)
x_train.shape, y_train.shape



((2800, 62500), (2800,))

In [22]:
model = tf.keras.Sequential([
    tf.keras.layers.Reshape((150,150,1), input_shape=(22500,)),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_1 (Reshape)         (None, 150, 150, 1)       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 148, 148, 32)      320       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 74, 74, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 global_average_pooling2d_1   (None, 64)               0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_2 (Dense)             (None, 64)               

In [23]:
history = model.fit(x_train, y_train, epochs=50, batch_size=32, validation_split=0.2)


Epoch 1/50


ValueError: in user code:

    File "c:\Users\joaog\OneDrive\Documentos\visao_computacional\venv\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\joaog\OneDrive\Documentos\visao_computacional\venv\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\joaog\OneDrive\Documentos\visao_computacional\venv\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\joaog\OneDrive\Documentos\visao_computacional\venv\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\joaog\OneDrive\Documentos\visao_computacional\venv\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None

In [ ]:
#pegar os dados de tests
test_path = './cat_dog_1/test'
test_files = [os.path.join(test_path, f) for f in os.listdir(test_path)]
test_imagens = []
classes_test = [] # 0 - gato, 1 - cachorro

for f in test_files:
    img = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (250, 250))
    img = img.ravel()
    test_imagens.append(img)
    class_name = os.path.basename(f).split('\\')[-1].split('.')[0]
    if 'cat' in class_name:
        classes_test.append(0)
    else:
        classes_test.append(1)

test_imagens = np.asarray(test_imagens)
classes_test = np.asarray(classes_test)
test_scaled = scaler.transform(test_imagens)
test_scaled = test_scaled.astype(np.float32)


test_imagens.shape, classes_test.shape

((1000, 22500), (1000,))

In [ ]:
predict = model.predict(test_imagens)
predict = (predict > 0.5).astype(bool) # true 1 - cachorro, false 0 - gato

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

accurrancy = accuracy_score(classes_test, predict)
confusion = confusion_matrix(classes_test, predict)
accurrancy

32/32 [==============================] - 0s 14ms/step


0.595